## LangChain tools
Tools w LangChain to zewnętrzne funkcje lub usługi (np. wyszukiwarka, kalkulator, API), które agent może wywoływać podczas rozwiązywania zadań, aby rozszerzyć swoje możliwości poza samo generowanie tekstu.

### Instalacja bibliotek

In [9]:
!pip install -q langchain langchain-openai python-dotenv

In [10]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Model gotowy.")


Model gotowy.


### Proste narzędzie (kalkulator)

In [11]:
from langchain.tools import tool

@tool
def multiply_numbers(x: int, y: int) -> int:
    """Mnoży dwie liczby całkowite."""
    return x * y

print(multiply_numbers.invoke({"x": 6, "y": 7}))


42


### Użycie narzędzia z LLM

In [12]:
from langchain_classic.agents import initialize_agent, AgentType

tools = [multiply_numbers]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Model sam zdecyduje, że musi użyć multiply_numbers
agent.run("Ile wynosi dwukrotność liczby ludzi na Ziemi?")


/tmp/ipykernel_19720/3483617261.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the [LangGraph documentation](https://langchain-ai.github.io/langgraph/) as well as guides for [Migrating from AgentExecutor](https://python.langchain.com/docs/how_to/migrate_agent/) and LangGraph's [Pre-built ReAct agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).
  agent = initialize_agent(
/tmp/ipykernel_19720/3483617261.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Ile wynosi dwukrotność liczby ludzi na Ziemi?")




> Entering new AgentExecutor chain...
Thought: Najpierw muszę ustalić aktualną liczbę ludzi na Ziemi. Szacuje się, że liczba ta wynosi około 8 miliardów. Następnie obliczę dwukrotność tej liczby.

Action:
```
{
  "action": "multiply_numbers",
  "action_input": {
    "x": 8000000000,
    "y": 2
  }
}
```

Observation: 16000000000
Thought:Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów."
}
```


> Finished chain.


'Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów.'

### Kolejne narzędzie (prosta baza wiedzy)

In [13]:
@tool
def company_info(name: str) -> str:
    """Zwraca podstawowe informacje o firmie na podstawie nazwy."""
    companies = {
        "OpenAI": "Firma zajmująca się sztuczną inteligencją, twórca ChatGPT.",
        "LangChain": "Biblioteka ułatwiająca budowanie aplikacji z modelami językowymi.",
        "Tesla": "Producent samochodów elektrycznych i baterii."
    }
    return companies.get(name, "Nie znam tej firmy.")

tools = [company_info]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Powiedz mi, czym zajmuje się firma Tesla.")




> Entering new AgentExecutor chain...
Muszę uzyskać podstawowe informacje o firmie Tesla, aby dowiedzieć się, czym się zajmuje.  
Action: company_info  
Action Input: "Tesla"  
Observation: Producent samochodów elektrycznych i baterii.
Thought:I now know the final answer  
Final Answer: Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.

> Finished chain.


'Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.'

### Dostępne tool w LangChain
Lista dostępnych narzędzi: https://python.langchain.com/docs/integrations/tools/

In [14]:
!pip install wikipedia tavily-python

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_classic.agents import initialize_agent, AgentType

# Inicjalizacja narzędzia wyszukiwarki
# klucz API do Tavily: https://app.tavily.com/home
search = TavilySearchResults(max_results=3)

# Bezpośrednie użycie narzędzia
query = "Najnowsze wiadomości o premierze Polski"
results = search.invoke(query)

print(f"Wyniki wyszukiwania dla zapytania: '{query}'")
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result.get('title', 'Brak tytułu')}")
    print(f"   URL: {result.get('url', 'Brak URL')}")
    print(f"   Treść: {result.get('content', 'Brak treści')[:200]}...")

# Lista narzędzi z wyszukiwarką
tools = [search]

# Agent z dostępem do internetu
agent_with_search = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Przykładowe zapytanie wymagające wyszukiwania w internecie
response = agent_with_search.run(
    "Kto jest obecnym premierem Polski i ile ma lat?"
)
print(f"\nOdpowiedź agenta:\n{response}")

Wyniki wyszukiwania dla zapytania: 'Najnowsze wiadomości o premierze Polski'

1. Premierzy Polski – Wikipedia, wolna encyklopedia
   URL: https://pl.wikipedia.org/wiki/Premierzy_Polski
   Treść: | (13.) |  | Donald Tusk (PO, od 25 października 2025 roku KO) (ur. 1957) | 11 grudnia 2023 | 13 grudnia 2023 | 12 grudnia 2023 11 czerwca 2025 |  |  | 692 dni | Trzeci rząd Donalda Tuska |
| Karol Na...

2. Premier - Kancelaria Prezesa Rady Ministrów - Portal Gov.pl
   URL: https://www.gov.pl/web/premier
   Treść: Premier: Uczciwi nie mają się czego bać
 28.10.2025  Białystok

  Premier: Podlasie jest dziś kluczowym miejscem dla bezpieczeństwa Polski
 28.10.2025

  Premier o aferze CPK: Smród korupcji wokół tej...

3. Prezes Rady Ministrów – Wikipedia, wolna encyklopedia
   URL: https://pl.wikipedia.org/wiki/Prezes_Rady_Ministr%C3%B3w
   Treść: Prezes Rady Ministrów

| Obecny Premier |
| Donald Tusk |
| Stanowisko |
| Państwo | Polska |
| Data utworzenia | 17 listopada 1918 |
| Pierwszy Premie

In [16]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Inicjalizacja narzędzia Wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Bezpośrednie użycie
wiki_result = wikipedia.run("Python programming language")
print(f"Wikipedia - Python:\n{wiki_result[:300]}...")

# Agent z wieloma narzędziami
# Agent z dostępem do wielu źródeł informacji
tools_extended = [search, wikipedia, company_info]

agent_multi_tool = initialize_agent(
    tools=tools_extended,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Zapytanie wymagające użycia różnych narzędzi
response = agent_multi_tool.run(
    "Znajdź informacje o Pythonie w Wikipedii, a następnie wyszukaj "
    "najnowsze artykuły o nowych funkcjach w Python 3.13"
)
print(f"\nOdpowiedź:\n{response}")

Wikipedia - Python:
Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, ...


> Entering new AgentExecutor chain...
Muszę najpierw znaleźć ogólne informacje o Pythonie w Wikipedii, a następnie poszukać najnowszych artykułów dotyczących nowych funkcji w Python 3.13. 

Action: wikipedia  
Action Input: Python (programming language)  
Observation: Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.
Guido van Ros